## 1. data load

In [ ]:
import requests
url = 'https://biendata.com/competition/airquality/ld/2018-03-01-0/2018-05-01-23/2k0d1d8'
respones= requests.get(url)
with open ("ld_airquality.csv",'w') as f:
    f.write(respones.text)
    
url = 'https://biendata.com/competition/meteorology/ld_grid/2018-03-01-0/2018-05-01-23/2k0d1d8'
respones= requests.get(url)
with open ("ld_grid_airquality.csv",'w') as f:
    f.write(respones.text)

In [1]:
import pandas as pd

ld_airquality = pd.read_csv("ld_airquality.csv")
ld_grid_airquality = pd.read_csv("ld_grid_airquality.csv")
ld_air_stations = pd.read_csv("London_AirQuality_Stations.csv")
ld_historical_aqi_stations = pd.read_csv("London_historical_aqi_forecast_stations_20180331.csv")
ld_historical_aqi_other_stations = pd.read_csv("London_historical_aqi_other_stations_20180331.csv")
ld_historical_meo_grid = pd.read_csv("London_historical_meo_grid.csv")

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 2. make empty dataframe ld

In [2]:
import datetime

ld = pd.DataFrame(columns=['station_id', 'utc_time'])
ld_airquality = ld_airquality.rename(columns={'time':'utc_time'})
ld_station_list = list(set(ld_airquality['station_id']))
utc_time = sorted(list(set(ld_airquality['utc_time'])))
start_time = datetime.datetime.strptime('2017-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
utc_time = [(start_time + datetime.timedelta(hours=i)).isoformat().replace('T', ' ') for i in range(600*24)]
ld['station_id'] = sorted(ld_station_list * 600 * 24)
ld['utc_time'] = utc_time * len(ld_station_list)

## 3. merge ld_historical data

In [3]:
def change_time_format(data):
    tmp_time = datetime.datetime.strptime(data, '%Y/%m/%d %H:%M')
    return tmp_time.isoformat().replace('T', ' ')

In [5]:
air = ld_historical_aqi_stations[['station_id','MeasurementDateGMT','PM2.5 (ug/m3)','PM10 (ug/m3)','NO2 (ug/m3)']]
air.columns = ['station_id','utc_time','PM2.5','PM10','NO2']

air2 = ld_historical_aqi_other_stations[['Station_ID','MeasurementDateGMT','PM2.5 (ug/m3)','PM10 (ug/m3)','NO2 (ug/m3)']]
air2 = air2.dropna()
air2.columns = ['station_id','utc_time','PM2.5','PM10','NO2']

print(air.shape)
print(air2.shape)

(141661, 5)
(27906, 5)


In [6]:
ld_historical_airquality = pd.concat([air,air2])
print(ld_historical_airquality.shape)
ld_historical_airquality['utc_time'] = list(map(change_time_format,ld_historical_airquality['utc_time']))

(169567, 5)


## 4. add ld_historical_airquality

In [7]:
ld_result = pd.merge(ld, ld_historical_airquality, on=['station_id','utc_time'])
ld_result.head()

,station_id,utc_time,PM2.5,PM10,NO2
0,BL0,2017-01-01 00:00:00,30.8,31.6,10.8
1,BL0,2017-01-01 01:00:00,22.9,34.6,17.3
2,BL0,2017-01-01 02:00:00,18.4,25.2,14.2
3,BL0,2017-01-01 03:00:00,19.5,24.0,14.8
4,BL0,2017-01-01 04:00:00,22.1,26.7,16.7


## 5. get nearest_grid

In [9]:
from utils import get_nearest_grid
ld_result['stationName'] = list(map(get_nearest_grid, ld_result['station_id']))

## 6. merge airquality & meo_grid

In [11]:
ld_result = pd.merge(ld_result, ld_historical_meo_grid, on=['stationName','utc_time'])
ld_result = ld_result.sort_values(by=['station_id','utc_time'])
ld_result.head()

,station_id,utc_time,PM2.5,PM10,NO2,stationName,longitude_x,latitude_x,temperature_x,pressure_x,humidity_x,wind_direction_x,wind_speed/kph_x,longitude_y,latitude_y,temperature_y,pressure_y,humidity_y,wind_direction_y,wind_speed/kph_y
0,BL0,2017-01-01 00:00:00,30.8,31.6,10.8,london_grid_409,-0.1,51.5,5.80,1019.93,91.38,221.79,15.57,-0.1,51.5,5.80,1019.93,91.38,221.79,15.57
2,BL0,2017-01-01 01:00:00,22.9,34.6,17.3,london_grid_409,-0.1,51.5,5.84,1018.73,89.96,220.34,16.49,-0.1,51.5,5.84,1018.73,89.96,220.34,16.49
4,BL0,2017-01-01 02:00:00,18.4,25.2,14.2,london_grid_409,-0.1,51.5,5.89,1017.53,88.54,219.03,17.41,-0.1,51.5,5.89,1017.53,88.54,219.03,17.41
6,BL0,2017-01-01 03:00:00,19.5,24.0,14.8,london_grid_409,-0.1,51.5,5.93,1016.33,87.12,217.86,18.35,-0.1,51.5,5.93,1016.33,87.12,217.86,18.35
8,BL0,2017-01-01 04:00:00,22.1,26.7,16.7,london_grid_409,-0.1,51.5,6.12,1015.38,87.54,216.18,18.78,-0.1,51.5,6.12,1015.38,87.54,216.18,18.78


In [12]:
ld_result.shape

(161541, 20)

## 7. dropna

In [13]:
ld_result = ld_result.dropna()

In [14]:
ld_result.shape

(118799, 20)

In [15]:
ld_result.to_csv("london_table.csv")